In [4]:
import os
import json
# import pyodbc
from datetime import datetime, timedelta

In [ ]:
class Custom_Topic:
    def __init__(self, custom_topic, keyword_score_list, total_score, start_date, end_date):
        self.custom_topic_id = self.get_custom_topic_id()
        self.custom_topic = custom_topic.lower()
        self.keyword_score = self.get_keyword_score(keyword_score_list)
        self.total_score = total_score
        self.start_date = datetime.strptime(start_date, '%m-%d-%Y').date()
        self.end_date = datetime.strptime(end_date, '%m-%d-%Y').date()
    
    def get_custom_topic_id(self) -> int:
        return -1
    
    def get_keyword_score(keyword_score_list: list) -> list(tuple):
        keyword_score = []
        keyword_score_list_len = len(keyword_score_list)
        if (keyword_score_list_len % 2 != 0):
            print('Missing score for a keyword')
            quit()
        for i in range(0, keyword_score_list_len, 2):
            if ('(' in keyword_score_list[i]):
                with_stop_word = keyword_score_list[i].replace('(', '')
                with_stop_word = with_stop_word.replace(')', '')
                keyword_score.append((with_stop_word.lower(), keyword_score_list[i + 1]))
                without_stop_word = re.sub(r'\(.*\)', '', keyword_score_list[i])
            else:
                keyword_score.append((keyword_score_list[i].lower(), keyword_score_list[i + 1]))
        return keyword_score
            
    


In [5]:
class Database:
    def __init__(self, env = 'prod'):
        with open(os.path.join(PATH_CONFIG, DB_CONFIG)) as file:
            database_info = json.load(file)
        self.conn_common = self._database_conn(database_info[env], 'common')
        self.conn_dmp = self._database_conn(database_info[env], 'dmp')

    def _database_conn(self, database_info, database):
        return 'DRIVER={};\
                    SERVER={};\
                    DATABASE={};\
                    UID={};\
                    PWD={};\
                    TrustServerCertificate=yes'.format(
            database_info['driver'], database_info['server'], 
            database_info['database'][database],
            database_info['username'], database_info['password']
        )

    def add_custom_topic(self, )

In [8]:
class Student:
    # Class variable
    school_name = 'ABC School '
    
    def __init__(self, name, roll_no):
        self.name = name
        self.roll_no = roll_no

# create first object
s1 = Student('Emma', 10)
print(s1.name, s1.roll_no, s1.school_name)
# access class variable

# create second object
s2 = Student('Jessa', 20)
# access class variable
print(s2.name, s2.roll_no, s2.school_name)

Emma 10 ABC School 
Jessa 20 ABC School 
